In [ ]:
import json
import os.path
import numpy as np
import xarray as xr
import geopandas as gpd

from xcube.core.store import new_data_store
from xcube.core.geom import rasterize_features
from xcube.core.geom import mask_dataset_by_geometry
from xcube.core.resampling.affine import affine_transform_dataset
from xcube.core.gridmapping import GridMapping

---
### Open Country Geometries

In [ ]:
with open('../resources/countries-50m.geojson') as fp:
    features = json.load(fp)
features

features = gpd.GeoDataFrame.from_features(features)
features.plot(figsize=(10, 10))

In [ ]:
#features

In [ ]:
desired_point = dict(lon=38., lat=0.)
desired_country = 'Kenya'

#desired_point = dict(lon=-4.75, lat=40.35)
#desired_country = 'Spain'

desired_feature = features.loc[features['sovereignt'] == desired_country]
desired_feature

In [ ]:
desired_geometry = desired_feature.iloc[0].geometry
desired_geometry

---
### Inspect CCI-Toolbox' Zarr Store (high performance data access)

In [ ]:
store = new_data_store('ccizarr')

In [ ]:
list(store.get_data_ids())

---
### Open Land Cover ECV

In [ ]:
lc_dataset = store.open_data('ESACCI-LC-L4-LCCS-Map-300m-P1Y-1992-2015-v2.0.7b.zarr')
lc_dataset

In [ ]:
lc_dataset.lccs_class

In [ ]:
lc_dataset.time

In [ ]:
lc_dataset_subset = mask_dataset_by_geometry(lc_dataset, desired_geometry, all_touched=True)

In [ ]:
lc_dataset_subset.lccs_class.isel(time=0).plot.imshow(cmap="tab20b")

---
### Open Soil Moisture ECV

In [ ]:
sm_dataset = store.open_data('ESACCI-SOILMOISTURE-L3S-SSMV-COMBINED-1978-2020-fv05.3.zarr')
sm_dataset

In [ ]:
sm_dataset.sm

In [ ]:
sm_dataset_subset = mask_dataset_by_geometry(sm_dataset, desired_geometry, all_touched=True)

In [ ]:
sm_dataset_subset_annual = sm_dataset_subset.resample(time='1Y').mean()

In [ ]:
sm_dataset_subset_annual.sm.time

In [ ]:
ds_name = f"sm_dataset_annual_{desired_country}.zarr"
if not os.path.exists(ds_name):
    sm_dataset_subset_annual.to_zarr(f"sm_dataset_annual_{desired_country}.zarr")
sm_dataset_subset_annual = xr.open_zarr(ds_name)

In [ ]:
sm_dataset_subset_annual.sm.isel(time=0).plot.imshow(cmap="Blues")

In [ ]:
sm_dataset_subset_annual.sm.sel(**desired_point, method='nearest').plot(figsize=(20, 4))

In [ ]:
sm_dataset_subset_annual.sm.mean(dim=['lat','lon']).plot(figsize=(20, 4))

---
### Combination

In [ ]:
desired_year = '2015-01-01'

In [ ]:
ds_name = f"lc_dataset_subset_slice_{desired_country}.zarr"
if not os.path.exists(ds_name):
    lc_dataset_subset_slice = lc_dataset_subset.sel(time=desired_year, method='nearest')
    lc_dataset_subset_slice = lc_dataset_subset_slice.chunk(dict(lon=2000, lat=2000))
    lc_dataset_subset_slice.to_zarr(ds_name)
lc_dataset_subset_slice = xr.open_zarr(ds_name)
lc_dataset_subset_slice

In [ ]:
lc = lc_dataset_subset_slice.lccs_class
lc

In [ ]:
lc.plot.imshow(cmap="tab20b", figsize=(12, 10))

In [ ]:
sm_dataset_subset_slice = sm_dataset_subset_annual.sel(time=desired_year, method='nearest')
sm = sm_dataset_subset_slice.sm
sm

In [ ]:
sm.plot.imshow(cmap='Blues', figsize=(12, 10))

In [ ]:
ds_name = f"sm_dataset_coreg_{desired_country}.zarr"
if not os.path.exists(ds_name):
    sm_dataset_coreg = affine_transform_dataset(
        sm_dataset_subset_slice, 
        GridMapping.from_dataset(sm_dataset_subset_slice),
        GridMapping.from_dataset(lc_dataset_subset_slice)
    )
    sm_dataset_coreg.to_zarr(ds_name)
sm_dataset_coreg = xr.open_zarr(ds_name)
sm = sm_dataset_coreg.sm
sm

In [ ]:
sm.plot.imshow(cmap='Blues', figsize=(12, 10))

In [ ]:
mask = (lc == 120) | (lc == 130)
mask

In [ ]:
mask.data.visualize()

In [ ]:
mask.plot.imshow(cmap='binary', figsize=(12, 10))

In [ ]:
mask.indexes

In [ ]:
sm.indexes

In [ ]:
ds = xr.Dataset(dict(sm=sm.drop("time"), lc=lc.drop("time")))
ds

In [ ]:
ds.sm.where(ds.lc == 110)

In [ ]:
import dask.array as da
sm_masked = da.where(mask.data, sm.data, np.nan)
sm_masked

In [ ]:
xr.DataArray(sm_masked, coords=sm.coords).plot.imshow(cmap='Blues', figsize=(12, 10))

In [ ]:
sm_masked.data.visualize()

In [ ]:
sm_masked.plot.imshow()

In [ ]:
sm_data = np.where(mask, sm, np.nan)
sm_data

In [ ]:
xr.DataArray(sm_data, coords=sm.coords).plot.imshow(cmap='Blues', figsize=(12, 10))

In [ ]:
lc_groups = lc.groupby(lc)
lc_groups

In [ ]:
lc_counts = lc_groups.count()


In [ ]:
lc_counts.plot()

In [ ]:
sorted([(int(item.lccs_class), int(item)) for item in lc_counts], key=lambda k: k[1], reverse=True) 

In [ ]:
lc.where(lc == 10).plot.imshow(figsize=(20,20))

In [ ]:
#sm_grouped_by_lc = sm.groupby(lc)

In [ ]:
#sm_grouped_by_lc

In [ ]:
#m_grouped_by_lc.count()

In [ ]:
list(zip(lc.attrs['flag_meanings'].split(' '), lc.attrs['flag_values']))

In [ ]:
lc_dataset_subset.lccs_class.where(lc_dataset_subset.lccs_class == 10).count(dim=('lat', 'lon')).plot()

In [ ]:
lc_dataset_subset.lccs_class.where(lc_dataset_subset.lccs_class == 110).count(dim=('lat', 'lon')).plot()

In [ ]:
lc_dataset_subset.lccs_class.where(lc_dataset_subset.lccs_class == 130).count(dim=('lat', 'lon')).plot()

In [ ]:
lc_dataset_subset.lccs_class.where(lc_dataset_subset.lccs_class == 40).count(dim=('lat', 'lon')).plot()

In [ ]:
lc_dataset_subset.lccs_class.where((lc_dataset_subset.lccs_class >= 10) & (lc_dataset_subset.lccs_class <= 12)).count(dim=('lat', 'lon')).plot()

In [ ]:
lc_dataset_subset.lccs_class.where(lc_dataset_subset.lccs_class == 20).count(dim=('lat', 'lon')).plot()

In [ ]:
lc_dataset_subset.lccs_class.where(lc_dataset_subset.lccs_class == 200).count(dim=('lat', 'lon')).plot()

In [ ]:
lc_dataset_subset.lccs_class.where(lc_dataset_subset.lccs_class > 0).count(dim=('lat', 'lon')).plot()